# Training Pipeline 

In [2]:
# Imports

import os
import sys
import torch
import ipywidgets as wg
from IPython.display import display, Javascript 
from argparse import ArgumentParser, Namespace

os.sys.path.append(os.path.dirname(os.path.abspath('.')))
from utils.jupyter_widgets import get_pipelin_widget, get_parameter_widgets, get_execution_widgets

In [3]:
# Check for GPU

use_cuda = torch.cuda.is_available()
if use_cuda:
    print('The following GPU was found:\n')
    print('CUDNN VERSION:', torch.backends.cudnn.version())
    print('Number CUDA Devices:', torch.cuda.device_count())
    print('CUDA Device Name:',torch.cuda.get_device_name(0))
    print('CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)
else:
    print('No GPU was found. CPU will be used.')

# Select a pipeline 
pipeline = get_pipelin_widget()
display(pipeline)

The following GPU was found:

CUDNN VERSION: 8200
Number CUDA Devices: 1
CUDA Device Name: NVIDIA RTX A4000
CUDA Device Total Memory [GB]: 16.890134528


Dropdown(description='Select Pipeline:', options=('DiffusionModel3d', 'DiffusionModel2d'), style=DescriptionSt…

---
After executing the next block, please adapt all parameters accordingly.
The pipeline expects lists of files that should be used for training, testing and validation. 
Absolute paths to each files are automatically obtained by concatenating the provided data root and each entry of the file lists.
When changing the selected pipeline, please again execute the following block.

In [7]:
# Define general training parameters
params = {'output_path': 'results/experiment_name',
          'log_path': 'logs',
          'gpus': use_cuda,
          'no_resume': False,
          'pretrained': None,
          'augmentations': None,
          'epochs': 5000,
          'pipeline': pipeline.value}

# Load selected pipeline
if   params['pipeline'].lower() == 'diffusionmodel3d':
    from models.DiffusionModel3D import DiffusionModel3D as network
elif params['pipeline'].lower() == 'diffusionmodel2d':
    from models.DiffusionModel2D import DiffusionModel2D as network
else:
    raise ValueError('Pipeline {0} unknown.'.format(params['pipeline']))
# Get and show corresponding parameters
pipeline_args = ArgumentParser(add_help=False)
pipeline_args = network.add_model_specific_args(pipeline_args)
pipeline_args = vars(pipeline_args.parse_known_args()[0])
params = {**params, **pipeline_args}

print('-'*60+'\nPARAMETER FOR PIPELINE "{0}"\n'.format(pipeline.value)+'-'*60)
param_names, widget_list, _ = get_parameter_widgets(params)
for widget in widget_list: 
    display(widget)
    
print('-'*60+'\nEXECUTION SETTINGS\n'+'-'*60)
wg_execute, wg_arguments = get_execution_widgets()
display(wg_arguments)
display(wg_execute)

------------------------------------------------------------
PARAMETER FOR PIPELINE "DiffusionModel3d"
------------------------------------------------------------


Text(value='results/experiment_name', description='Output Path:', style=DescriptionStyle(description_width='in…

Text(value='logs', description='Log Path:', style=DescriptionStyle(description_width='initial'))

IntSlider(value=1, description='Use GPU:', max=1, style=SliderStyle(description_width='initial'))

Checkbox(value=True, description='Resume:', style=DescriptionStyle(description_width='initial'))

...

BoundedIntText(value=1, description='Batch Size:', max=1000000, min=1, style=DescriptionStyle(description_widt…

BoundedFloatText(value=0.001, description='Learning Rate:', max=1000000.0, style=DescriptionStyle(description_…

------------------------------------------------------------
EXECUTION SETTINGS
------------------------------------------------------------


Checkbox(value=True, description='Get Command Line Arguments', style=DescriptionStyle(description_width='initi…

Checkbox(value=True, description='Execute Now!', style=DescriptionStyle(description_width='initial'))

---
Finish preparations and start processing by executing the next block.

In [ ]:
# Get parameters
param_names = [p for p,w in zip(param_names, widget_list) if w.value!=False and w.value!='']
widget_list = [w for w in widget_list if w.value!=False and w.value!='']
command_line_args = ' '.join(['--pipeline {0}'.format(pipeline.value)]+[n+' '+str(w.value) if not type(w.value)==bool else n for n,w in zip(param_names, widget_list)])

# Show the command line arguments
if wg_arguments.value:
    print('_'*60+'\nCOMMAND LINE ARGUMENTS FOR PIPELINE "{0}"\n'.format(pipeline.value)+'-'*60)
    print(command_line_args)
    print('\n')
    
# Execute the pipeline
if wg_execute.value:
    print('_'*60+'\nEXECUTING PIPELINE "{0}"\n'.format(pipeline.value)+'-'*60)
    %run "../train_script.py" {command_line_args}